In [ ]:
# -*- coding: utf-8 -*-
"""
Model 2 (主结果): SEM · log–log  （KNN=6，行标准化）
  ln(Y_it) = α + β ln(VET_it) + e_it
  e_it     = λ W e_it + μ_it

Y:   log_gdp_pc (已对数), employment_rate, unemployment_rate (转 0–1 后取 ln)
X:   ln_vet = ln(vet_per_million)
W:   KNN(k=6)（row-standardized）

输出(逐年):
- OLS_loglog_*_2014_2023.csv          # 基线 OLS + Moran(OLS 残差)
- SEM_loglog_*_X_ln_vet_2014_2023.csv # SEM：β/SE/p, λ/SE/p, logLik, AIC, n
- MORAN_SEM_RESID_*_2014_2023.csv     # SEM 残差 Moran’s I
- ./<Y>/SEM_<Y>_<year>.txt            # 每年 SEM 文本摘要
"""

import warnings, os
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.api as sm
from scipy.stats import chi2
from esda.moran import Moran
from libpysal.weights import KNN
from spreg import ML_Error

# ── 路径 ───────────────────────────────────────────────────────────────
PATH_GEO = r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\cleaned geo data\NUTS2_2021.geojson"
PATH_PANEL = r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\panel_long_merged_with_general.csv"
OUT_DIR    = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\moran\SEM_fff")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 选项 ───────────────────────────────────────────────────────────────
YEARS         = list(range(2014, 2024))
ADD_CONST     = True
KNN_K         = 6
PERMUTATIONS  = 999
EPS           = 1e-9
ZERO_POLICY   = "drop"     # 就业/失业率: 'drop' 丢 <=0 后取 ln；或 'eps' 用 ln(y+EPS)
USE_LR_BACKUP = True       # λ 的 p 值：如拿不到 Wald，就用 LR(1) 兜底

Y_LIST = [
    ("log_gdp_pc",        "log GDP per capita (already log)"),
    ("employment_rate",   "Employment rate"),
]

# ── 工具函数 ─────────────────────────────────────────────────────────────
def clean_cols(df):
    df.columns = (df.columns.astype(str)
                  .str.replace("\ufeff","", regex=False)
                  .str.strip().str.lower()
                  .str.replace(r"\s+","_", regex=True))
    return df

def detect_geo_id(gdf: gpd.GeoDataFrame) -> str:
    for c in ("nuts_id","NUTS_ID","nuts_id_2021","id","geo","region","code"):
        if c in gdf.columns: return c
    return [c for c in gdf.columns if c != gdf.geometry.name][0]

def detect_panel_id(df: pd.DataFrame) -> str:
    for c in ("region","region_code","nuts_id","id","geo","code"):
        if c in df.columns: return c
    raise ValueError("panel 表缺少地区ID列（region/region_code/nuts_id/id/...）")

def pct_to01(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    return s/100.0 if s.dropna().max() > 1.0000001 else s

def parse_zp(z_entry):
    """兼容 spreg 不同版本，从 z_stat 项中取 (z, p)。"""
    try:
        if isinstance(z_entry, (list, tuple)):
            if len(z_entry)==2 and isinstance(z_entry[1], (list, tuple, np.ndarray)):
                return float(z_entry[1][0]), float(z_entry[1][1])  # ('name',(z,p))
            if len(z_entry)>=2 and np.isscalar(z_entry[0]) and np.isscalar(z_entry[1]):
                return float(z_entry[0]), float(z_entry[1])        # (z,p)
            if len(z_entry)>=3 and np.isscalar(z_entry[1]) and np.isscalar(z_entry[2]):
                return float(z_entry[1]), float(z_entry[2])        # ('name',z,p)
        return float(z_entry), float("nan")
    except Exception:
        return float("nan"), float("nan")

# ── 读数据 ───────────────────────────────────────────────────────────────
g  = gpd.read_file(PATH_GEO)
df = pd.read_csv(PATH_PANEL, encoding="utf-8-sig")
g, df = clean_cols(g), clean_cols(df)

GEO_ID   = detect_geo_id(g)
PANEL_ID = detect_panel_id(df)

g[GEO_ID]    = g[GEO_ID].astype(str).str.strip().str.upper()
df[PANEL_ID] = df[PANEL_ID].astype(str).str.strip().str.upper()
g = g[g[GEO_ID].isin(set(df[PANEL_ID]))].drop_duplicates(subset=[GEO_ID]).copy()

# 投影(若经纬度则转 LAEA Europe EPSG:3035)
try:
    if (g.crs is None) or g.crs.to_epsg() == 4326:
        g = g.to_crs(3035)
except Exception:
    pass

# 数值化
for c in ("year","log_gdp_pc","employment_rate", "general_per_million"):
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ln(VET)
if "general_per_million" not in df.columns:
    raise ValueError("缺少 general_per_million 列，无法计算 ln_general")
df["ln_general"] = np.log(df["general_per_million"].astype(float) + EPS)

# ── 主循环 ───────────────────────────────────────────────────────────────
for y_name, y_nice in Y_LIST:
    print(f"\n=== Outcome: {y_name} — {y_nice}; X = ln_general; W = KNN(k={KNN_K}) ===")
    rows_ols, rows_sem, rows_mi_sem = [], [], []

    for yr in YEARS:
        # ln(Y)
        sub = df.loc[df["year"]==yr, [PANEL_ID, "ln_general", y_name]].copy()
        if y_name in ("employment_rate"):
            y0 = pct_to01(sub[y_name])
            if ZERO_POLICY == "drop":
                y0 = y0.mask(y0 <= 0)
                lnY = np.log(y0)
            else:
                lnY = np.log(y0 + EPS)
        else:
            lnY = pd.to_numeric(sub[y_name], errors="coerce")  # log_gdp_pc 已是对数
        sub["ln_y"] = lnY
        sub = sub.dropna(subset=["ln_general","ln_y"]).copy()
        if len(sub) < 10:
            print(f"[{yr}] too few obs (n={len(sub)}), skip."); 
            continue

        # 对齐几何
        gg = g[[GEO_ID,"geometry"]].merge(
            sub[[PANEL_ID,"ln_general","ln_y"]].rename(columns={PANEL_ID:GEO_ID}),
            on=GEO_ID, how="inner"
        ).dropna()
        if len(gg) < 10:
            print(f"[{yr}] too few after merge (n={len(gg)}), skip."); 
            continue

        # 权重：KNN(k=6), row-standardized
        Wgt = KNN.from_dataframe(gg, k=KNN_K, use_index=True)
        Wgt.transform = "R"

        # OLS (log–log) + Moran(OLS 残差)
               # OLS (log–log) + Moran(OLS 残差)
        y_arr = gg["ln_y"].to_numpy(float).reshape(-1,1)
        x1    = gg["ln_general"].to_numpy(float).reshape(-1,1)
        X_arr = np.column_stack([np.ones((len(gg),1)), x1]) if ADD_CONST else x1

        ols = sm.OLS(y_arr.ravel(), X_arr).fit()
        coef_idx = 1 if ADD_CONST else 0  # ln_general 在参数向量中的位置
        beta_ols = float(ols.params[coef_idx])
        se_ols   = float(ols.bse[coef_idx])
        t_ols    = float(ols.tvalues[coef_idx])
        p_ols    = float(ols.pvalues[coef_idx])

        mi_ols = Moran(ols.resid, Wgt, permutations=PERMUTATIONS)

        rows_ols.append({
            "year": yr, "Y": y_name,
            "beta_ln_general": beta_ols,        # OLS 系数
            "se_ln_general": se_ols,            # OLS 标准误
            "t_ln_general": t_ols,              # OLS t 值（可选）
            "p_ln_general": p_ols,              # OLS p 值  ← 新增
            "R2": float(ols.rsquared),
            "Moran_I_resid": float(mi_ols.I),
            "Moran_p": float(mi_ols.p_sim),
            "llf_OLS": float(ols.llf),
            "n": int(len(gg))
        })


        # SEM (ML_Error)
        sem = ML_Error(
            y_arr, X_arr, w=Wgt,
            name_y=f"ln({y_name})", name_x=(["const","ln_vet"] if ADD_CONST else ["ln_vet"]),
            name_w=f"KNN(k={KNN_K})", name_ds=f"year={yr}"
        )

        # 取系数
        betas = np.asarray(sem.betas).ravel()
        beta  = float(betas[1]) if ADD_CONST else float(betas[0])
        lam   = float(np.asarray(sem.lam).ravel()[0])
        logL  = float(getattr(sem, "logll", np.nan))
        AIC   = float(getattr(sem, "aic", np.nan)) if hasattr(sem, "aic") else float(getattr(sem, "akaike", np.nan))

        # Wald z/p —— 兼容不同版本
        def _zp(idx):
            try: return parse_zp(sem.z_stat[idx])
            except Exception: return (np.nan, np.nan)
        z_beta, p_beta = _zp(1 if ADD_CONST else 0)
        z_lam,  p_lam  = _zp(-1)

        beta_se   = abs(beta / z_beta) if np.isfinite(z_beta) and z_beta!=0 else np.nan
        lambda_se = abs(lam  / z_lam ) if np.isfinite(z_lam)  and z_lam !=0 else np.nan

        # λ 的 LR(1) 兜底
        if USE_LR_BACKUP and not np.isfinite(p_lam):
            LR = max(0.0, 2*(logL - ols.llf))
            p_lam = float(chi2.sf(LR, 1))

        rows_sem.append({
            "year": yr, "Y": y_name,
            "beta_ln_general": beta, "se_ln_general": beta_se, "p_ln_general": p_beta,
            "lambda": lam, "lambda_se": lambda_se, "p_lambda": p_lam,
            "logLik": logL, "AIC": AIC, "n": int(len(gg))
        })

        # --- SEM 残差 Moran（正确做法：过滤后的 ε̂） ---
# u_hat：空间相关的误差；优先用 sem.u，取不到再用 y - predy 兜底
        try:
            u_hat = np.asarray(sem.u).ravel()
        except Exception:
            u_hat = (y_arr.ravel() - np.asarray(sem.predy).ravel())

        lam = float(np.asarray(sem.lam).ravel()[0])     # λ
        eps_hat = u_hat - lam * Wgt.sparse.dot(u_hat)   # ε̂ = (I - λW)û

        mi_sem = Moran(eps_hat, Wgt, permutations=PERMUTATIONS)
        rows_mi_sem.append({
            "year": yr, "Y": y_name,
            "I": float(mi_sem.I), "p": float(mi_sem.p_sim), "n": int(len(gg))
        })

        # 文本摘要
        (OUT_DIR / f"{y_name}").mkdir(parents=True, exist_ok=True)
        with open(OUT_DIR / f"{y_name}" / f"SEM_{y_name}_{yr}.txt", "w", encoding="utf-8") as f:
            f.write(str(sem.summary))

        print(f"[{y_name}] {yr}: n={len(gg)}, β={beta:.4f}, λ={lam:.4f}")

    # 保存
    pd.DataFrame(rows_ols).sort_values("year").to_csv(
        OUT_DIR / f"OLS_loglog_{y_name}_2014_2023general.csv", index=False, encoding="utf-8-sig")
    pd.DataFrame(rows_sem).sort_values("year").to_csv(
        OUT_DIR / f"SEM_loglog_{y_name}_X_ln_vet_2014_2023general.csv", index=False, encoding="utf-8-sig")
    pd.DataFrame(rows_mi_sem).sort_values("year").to_csv(
        OUT_DIR / f"MORAN_SEM_RESID_{y_name}_2014_2023general.csv", index=False, encoding="utf-8-sig")

print("\n✅ 完成。输出目录：", OUT_DIR)


=== Outcome: log_gdp_pc — log GDP per capita (already log); X = ln_vet; W = KNN(k=6) ===
[log_gdp_pc] 2014: n=187, β=0.1338, λ=0.8703
[log_gdp_pc] 2015: n=187, β=0.1493, λ=0.8661
[log_gdp_pc] 2016: n=187, β=0.1664, λ=0.8653
[log_gdp_pc] 2017: n=187, β=0.1981, λ=0.8612
[log_gdp_pc] 2018: n=192, β=0.3044, λ=0.8554
[log_gdp_pc] 2019: n=192, β=0.3693, λ=0.8512
[log_gdp_pc] 2020: n=185, β=0.4227, λ=0.8550
[log_gdp_pc] 2021: n=188, β=0.4452, λ=0.8566
[log_gdp_pc] 2022: n=186, β=0.4442, λ=0.8431
[log_gdp_pc] 2023: n=186, β=0.3951, λ=0.8335

=== Outcome: employment_rate — Employment rate; X = ln_vet; W = KNN(k=6) ===
[employment_rate] 2014: n=192, β=-0.0626, λ=0.8565
[employment_rate] 2015: n=192, β=-0.0493, λ=0.8563
[employment_rate] 2016: n=192, β=-0.0388, λ=0.8480
[employment_rate] 2017: n=192, β=-0.0343, λ=0.8439
[employment_rate] 2018: n=197, β=-0.0233, λ=0.8512
[employment_rate] 2019: n=197, β=-0.0239, λ=0.8470
[employment_rate] 2020: n=197, β=-0.0294, λ=0.8567
[employment_rate] 2021: n

In [2]:
import pandas as pd

df = pd.read_csv(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\panel_long_merged_with_general.csv")
print(df["vet_per_million"].agg(["mean", "std", "min", "max", "count"]))


mean     20726.734848
std       6603.955232
min       7351.446450
max      67498.273140
count     2145.000000
Name: vet_per_million, dtype: float64
